In [1]:
import os
import sys
sys.path.append(os.path.dirname((os.path.abspath(''))))

from src.pipeline import Pipeline
from src.data.preprocessing import removal, tokenization, stemming

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
pipeline = Pipeline()

In [3]:
pipeline.setup()

In [4]:
pipeline.preprocess()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 999.75it/s]


In [5]:
pipeline.create_BM25_features()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 248/248 [00:00<00:00, 2185.00it/s]


In [6]:
pipeline.create_tfidf_embeddings()

C:\Users\Admin\anaconda3\envs\InRe\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 6006.16it/s]


In [7]:
pipeline.create_tfidf_feature()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 248/248 [00:00<00:00, 1048.62it/s]


In [8]:
pipeline.create_glove_embeddings()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 3003.08it/s]


In [9]:
pipeline.create_glove_feature()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 248/248 [00:00<00:00, 2185.02it/s]


In [10]:
pipeline.create_jaccard_feature()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 248/248 [00:00<00:00, 2384.53it/s]


In [11]:
pipeline.create_sentence_features()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 248/248 [00:00<00:00, 62004.49it/s]


In [12]:
pipeline.create_interpretation_features()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 248/248 [00:00<00:00, 2917.65it/s]


In [13]:
pipeline.create_POS_features()

  0%|▊                                                                                                                                                                                         | 1/248 [00:00<00:02, 100.04it/s]


LookupError: 
**********************************************************************
  Resource [93maveraged_perceptron_tagger[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('averaged_perceptron_tagger')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtaggers/averaged_perceptron_tagger/averaged_perceptron_tagger.pickle[0m

  Searched in:
    - 'C:\\Users\\Admin/nltk_data'
    - 'C:\\Users\\Admin\\anaconda3\\envs\\InRe\\nltk_data'
    - 'C:\\Users\\Admin\\anaconda3\\envs\\InRe\\share\\nltk_data'
    - 'C:\\Users\\Admin\\anaconda3\\envs\\InRe\\lib\\nltk_data'
    - 'C:\\Users\\Admin\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [14]:
pipeline.create_interpretation_features()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 248/248 [00:00<00:00, 2818.15it/s]


In [15]:
pipeline.features

,qID,pID,y,bm25,tfidf_cosine,tfidf_euclidean,tfidf_manhattan,glove_cosine,glove_euclidean,glove_manhattan,...,words_difference,words_rel_difference,char_doc,char_query,char_difference,char_rel_difference,subjectivity_doc,polarity_doc,subjectivity_query,polarity_query
0,1164200,170065,1,-18.775658,0.381322,1.112365,4.352891,0.586853,70.378113,499.152158,...,68.0,12.333333,286,27,259.0,10.592593,0.170000,0.010000,0.40,0.00
2,407452,225591,1,-24.908448,0.512338,0.987585,5.206019,0.529600,71.512199,495.210797,...,61.0,16.250000,228,19,209.0,12.000000,0.450000,0.083333,0.25,0.25
3,29916,180662,1,-18.491710,0.450763,1.048081,4.490268,0.761738,61.558590,427.265441,...,58.0,20.333333,270,28,242.0,9.642857,0.312500,0.125000,0.00,0.00
4,537796,147073,1,-17.842617,0.391557,1.103126,4.994714,0.864105,125.005997,894.222114,...,98.0,17.333333,451,28,423.0,16.107143,0.571786,0.104643,0.00,0.00
5,1173738,147312,1,-18.372018,0.607108,0.886445,4.388424,0.874166,116.034302,758.357273,...,93.0,19.600000,436,21,415.0,20.761905,0.800000,0.400000,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,726150,231451,0,-10.551525,0.464992,1.034416,4.863496,0.107741,50.874332,336.001393,...,47.0,16.666667,256,9,247.0,28.444444,0.288000,0.192000,0.00,0.00
139,1183714,50234,0,-16.717235,0.313235,1.171977,4.608748,0.906084,111.312019,715.893759,...,61.0,13.200000,298,27,271.0,11.037037,0.348299,0.510204,0.40,0.10
140,711055,204455,0,-3.445145,0.025934,1.395755,5.312622,0.423969,156.802322,1216.671574,...,101.0,15.428571,473,30,443.0,15.766667,0.420833,0.025000,0.00,0.00
141,711055,78452,0,-3.475788,0.033585,1.390262,6.452086,0.379952,132.393494,979.556885,...,112.0,17.000000,548,30,518.0,18.266667,0.264921,0.051746,0.00,0.00
